In [ ]:
import pandas as pd
import numpy as np
import subscribers_module_live as sub
from datetime import datetime

import re

import sys
sys.path.append('../../zoho_data_app/')

# importing
import main_module as m
import zoho_filter_module as fil

%load_ext autoreload
%autoreload 2

In [ ]:
list = sub.load_lists_id() # load list id

maillwizz_cols = ['subscriber_uid',	
'CREATEDTIME',
'CREATEDDATE',	
'FNAME',	
'LNAME',	
'EMAIL',	
'PHONE',	
'MOBILE',	
'BIRTHDATE',	
'BIRTHYEAR',
'STREET',	
'STREET2',	
'CITY',	
'STATE',	
'COUNTRY',	
'DEPDATE',	
'ARRDATE',	
'CARDTYPE',	
'MARSTATUS',	
'LABELBRAND',	
'LEADSUBBRAND',	
'LEADSOURCE',	
'LEADLOCATIONS',	
'LEADSOURCEDESC',	
'LEADREGIONS',
'LEADSTATUS',
'EMAILOPTOUT',	
'OPTIN']

zoho_cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In']

dir_path = r'C:\Users\fajar\Documents\Python\Data\Zoho_Backup'
file_name = '\Leads_'

In [ ]:
load_maillwizz_df = sub.load_subs(list['Test'])
load_maillwizz_df = sub.reindex_cols(load_maillwizz_df, maillwizz_cols)
count_load_data_api = load_maillwizz_df.shape[0]
load_maillwizz_df

# load_maillwizz_df.to_csv(r'C:\Users\fajar\Documents\Python\Data\load_maillwizz_df.csv')

In [ ]:
# define data yang ingin ditemukan di list data booking dan data member 
path = r'C:\Users\fajar\Documents\Python\Data\Mailwizz Data\email-blacklist.csv'
mailwizz_blacklist_df = pd.read_csv(path, dtype='string', usecols=['Email'])
mailwizz_blacklist_df

In [ ]:
df = m.load_data_multiple(dir_path, file_name, zoho_cols)
count_data_awal = df.shape[0]
df = df.reindex(columns=zoho_cols)
df


In [ ]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

In [ ]:
zoho_filter = fil.filter_1(df)
zoho_df = df.loc[zoho_filter].copy() # menggunakan copy untuk menghindari setting copy warning
zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin

zoho_df.fillna('empty', inplace=True)
zoho_df

In [ ]:
# membersihkan format phone number dan email

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')

count_deleted_wrong_fromat = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

In [ ]:
# membersihkan email kosong dan duplikat

count_empty_email = zoho_df.loc[zoho_df['Email'] == 'empty'].shape[0]
zoho_df.drop(zoho_df.loc[zoho_df['Email'] == 'empty'].index, inplace=True)
count_after_dedup_empty_email = zoho_df.shape[0]

count_email_duplicate = zoho_df.loc[zoho_df['Email'].duplicated()].shape[0]
zoho_df.drop_duplicates('Email', inplace=True)
count_after_dedup_duplicated_email = zoho_df.shape[0]

zoho_df.reset_index(drop=True, inplace=True)
zoho_df

In [ ]:
# Cek Email OTA
count_email_OTA = m.find_OTA(zoho_df, 'Email')
zoho_df.loc[zoho_df['Email']=='OTA']

In [ ]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
zoho_df

In [ ]:
# find_member_booking = zoho_df.copy()

# find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
# find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
# find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
# find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
# find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

# find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
# find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

# find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

# find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

# count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
# count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

# find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.csv', index=False)

In [ ]:
# data phone, mobile dam email member yang ada di master data (wash member)
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]

In [ ]:
zoho_df.replace('empty', '', inplace=True)

In [ ]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Keseluruhan Master Zoho : "+str(count_data_awal))
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_fromat))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Email Kosong : "+str(count_empty_email))
print("Jumlah Setelah Clean Email Kosong : "+str(count_after_dedup_empty_email))
print("Jumlah Email Duplikat : "+str(count_email_duplicate))
print("Jumlah Setelah Clean Format Email Duplikat : "+str(count_after_dedup_duplicated_email))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Wash Member dan Booking : "+str(count_wash_member))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))

In [ ]:
cut_zoho_df = zoho_df.iloc[:15000].copy()
count_input_email_api = cut_zoho_df.shape[0]

cut_zoho_df

In [ ]:
# cut_zoho_df.drop(cut_zoho_df.loc[cut_zoho_df['Email'] == 'agathegillet@hotmail.com'].index, inplace=True)
# cut_zoho_df.reset_index(drop=True, inplace=True)
# cut_zoho_df.loc[cut_zoho_df['Email'] == 'agathegillet@hotmail.com']

# cut_zoho_df

# load_maillwizz_df.loc[load_maillwizz_df['EMAIL'] == 'agathegillet@hotmail.com']

In [ ]:
# input_new_subs_zoho.loc[input_new_subs_zoho['Messages'] == 'Input email success']
# input_new_subs_zoho.loc[input_new_subs_zoho['Messages'] == 'Error']
# deleted_mailwizz_df
# cut_zoho_df

In [ ]:
# cut_zoho_df['Created Time'] = 'Created Time'
# cut_zoho_df['Created Date']	= 'Created Date'
# cut_zoho_df['First Name'] = 'First Name'
# cut_zoho_df['Last Name'] = 'Last Name'
# cut_zoho_df['Phone'] = 'Phone'
# cut_zoho_df['Mobile'] = 'Mobile'
# cut_zoho_df['Birthdate'] = 'Birthdate'
# cut_zoho_df['Year of Birth'] = 'Year of Birth'
# cut_zoho_df['Street'] = 'Street'
# cut_zoho_df['Street2']	= 'Street2'
# cut_zoho_df['City'] = 'City'
# cut_zoho_df['State'] = 'State'	
# cut_zoho_df['Country'] = 'Country'
# cut_zoho_df['Departure Date'] = 'Departure Date'
# cut_zoho_df['Arrival Date'] = 'Arrival Date'
# cut_zoho_df['Card Type'] = 'Card Type'
# cut_zoho_df['Marital Status'] = 'Marital Status'
# cut_zoho_df['Lead Brand'] = 'Lead Brand'
# cut_zoho_df['Lead Sub-Brand'] = 'Lead Sub-Brand'
# cut_zoho_df['Lead Source'] = 'Lead Source'
# cut_zoho_df['Lead Locations'] = 'Lead Locations'
# cut_zoho_df['Lead Source Description'] = 'Lead Source Description'
# cut_zoho_df['Lead Regions'] = 'Lead Regions'
# cut_zoho_df['Lead Status'] = 'Lead Status'
# cut_zoho_df['Email Opt Out'] = 'Email Opt Out'
# cut_zoho_df['Opt In'] = 'Opt In'

# cut_zoho_df.to_csv(r'C:\Users\fajar\Documents\Python\Data\cutzohodf-test-dummy-update.csv')

In [ ]:
diff_field_email = sub.sync_maillwizz_zoho_field(load_maillwizz_df, 'EMAIL', cut_zoho_df, 'Email')

# menghapus row data merge yang hasil mergenya isna (untuk memfiltrasi data email mailwizz yang exist saja di list email zoho)
diff_field_email.drop(diff_field_email.loc[diff_field_email['Record Id'].isna()].index, inplace=True)
diff_field_email.reset_index(drop=True, inplace=True)

# hitung list email zoho yang ada di list email maillwizz
count_email_mailwizz_exist = diff_field_email.shape[0]

# # mencari list email mailwizz yang tidak ada di list email zoho
# maillwizz_exist_zoho_not_exist = diff_field_email.loc[diff_field_email['Record Id'].isna()]

diff_field_email.loc[
(diff_field_email['CREATEDTIME'] != diff_field_email['Created Time']) |
(diff_field_email['CREATEDDATE']	!= diff_field_email['Created Date']) |
(diff_field_email['FNAME'] != diff_field_email['First Name']) |
(diff_field_email['LNAME'] != diff_field_email['Last Name']) |
(diff_field_email['PHONE'] != diff_field_email['Phone']) |
(diff_field_email['MOBILE'] != diff_field_email['Mobile']) |
(diff_field_email['BIRTHDATE'] != diff_field_email['Birthdate']	) |
(diff_field_email['BIRTHYEAR'] != diff_field_email['Year of Birth']) |
(diff_field_email['STREET'] != diff_field_email['Street']) |
(diff_field_email['STREET2']	!= diff_field_email['Street2']) |
(diff_field_email['CITY'] != diff_field_email['City']) |
(diff_field_email['STATE'] != diff_field_email['State']) |	
(diff_field_email['COUNTRY'] != diff_field_email['Country']) |
(diff_field_email['DEPDATE'] != diff_field_email['Departure Date']) |
(diff_field_email['ARRDATE'] != diff_field_email['Arrival Date']) |
(diff_field_email['CARDTYPE'] != diff_field_email['Card Type']) |
(diff_field_email['MARSTATUS'] != diff_field_email['Marital Status']) |
(diff_field_email['LABELBRAND'] != diff_field_email['Lead Brand']) |
(diff_field_email['LEADSUBBRAND'] != diff_field_email['Lead Sub-Brand']) |
(diff_field_email['LEADSOURCE'] != diff_field_email['Lead Source']) |
(diff_field_email['LEADLOCATIONS'] != diff_field_email['Lead Locations']) |
(diff_field_email['LEADSOURCEDESC'] != diff_field_email['Lead Source Description']) |
(diff_field_email['LEADREGIONS'] != diff_field_email['Lead Regions']) |
(diff_field_email['LEADSTATUS'] != diff_field_email['Lead Status']) |
(diff_field_email['EMAILOPTOUT'] != diff_field_email['Email Opt Out']) |
(diff_field_email['OPTIN'] != diff_field_email['Opt In'])
]

# diff_field_email.to_csv(r'C:\Users\fajar\Documents\Python\Data\diif_field_email.csv')

In [ ]:
updated_subs = diff_field_email.loc[
    (diff_field_email['CREATEDTIME'] != diff_field_email['Created Time']) |
    (diff_field_email['CREATEDDATE']	!= diff_field_email['Created Date']) |
    (diff_field_email['FNAME'] != diff_field_email['First Name']) |
    (diff_field_email['LNAME'] != diff_field_email['Last Name']) |
    (diff_field_email['PHONE'] != diff_field_email['Phone']) |
    (diff_field_email['MOBILE'] != diff_field_email['Mobile']) |
    (diff_field_email['BIRTHDATE'] != diff_field_email['Birthdate']	) |
    (diff_field_email['BIRTHYEAR'] != diff_field_email['Year of Birth']) |
    (diff_field_email['STREET'] != diff_field_email['Street']) |
    (diff_field_email['STREET2']	!= diff_field_email['Street2']) |
    (diff_field_email['CITY'] != diff_field_email['City']) |
    (diff_field_email['STATE'] != diff_field_email['State']) |	
    (diff_field_email['COUNTRY'] != diff_field_email['Country']) |
    (diff_field_email['DEPDATE'] != diff_field_email['Departure Date']) |
    (diff_field_email['ARRDATE'] != diff_field_email['Arrival Date']) |
    (diff_field_email['CARDTYPE'] != diff_field_email['Card Type']) |
    (diff_field_email['MARSTATUS'] != diff_field_email['Marital Status']) |
    (diff_field_email['LABELBRAND'] != diff_field_email['Lead Brand']) |
    (diff_field_email['LEADSUBBRAND'] != diff_field_email['Lead Sub-Brand']) |
    (diff_field_email['LEADSOURCE'] != diff_field_email['Lead Source']) |
    (diff_field_email['LEADLOCATIONS'] != diff_field_email['Lead Locations']) |
    (diff_field_email['LEADSOURCEDESC'] != diff_field_email['Lead Source Description']) |
    (diff_field_email['LEADREGIONS'] != diff_field_email['Lead Regions']) |
    (diff_field_email['LEADSTATUS'] != diff_field_email['Lead Status']) |
    (diff_field_email['EMAILOPTOUT'] != diff_field_email['Email Opt Out']) |
    (diff_field_email['OPTIN'] != diff_field_email['Opt In'])
    ].copy()

count_email_maillwizz_updated = updated_subs.shape[0]

updated_subs['CREATEDTIME'] = updated_subs['Created Time']
updated_subs['CREATEDDATE']	= updated_subs['Created Date']
updated_subs['FNAME'] = updated_subs['First Name']
updated_subs['LNAME'] = updated_subs['Last Name']
updated_subs['PHONE'] = updated_subs['Phone'] 
updated_subs['MOBILE'] = updated_subs['Mobile'] 
updated_subs['BIRTHDATE'] = updated_subs['Birthdate']	 
updated_subs['BIRTHYEAR'] = updated_subs['Year of Birth'] 
updated_subs['STREET'] = updated_subs['Street'] 
updated_subs['STREET2']	= updated_subs['Street2'] 
updated_subs['CITY'] = updated_subs['City'] 
updated_subs['STATE'] = updated_subs['State'] 	
updated_subs['COUNTRY'] = updated_subs['Country'] 
updated_subs['DEPDATE'] = updated_subs['Departure Date'] 
updated_subs['ARRDATE'] = updated_subs['Arrival Date'] 
updated_subs['CARDTYPE'] = updated_subs['Card Type'] 
updated_subs['MARSTATUS'] = updated_subs['Marital Status'] 
updated_subs['LABELBRAND'] = updated_subs['Lead Brand'] 
updated_subs['LEADSUBBRAND'] = updated_subs['Lead Sub-Brand'] 
updated_subs['LEADSOURCE'] = updated_subs['Lead Source'] 
updated_subs['LEADLOCATIONS'] = updated_subs['Lead Locations'] 
updated_subs['LEADSOURCEDESC'] = updated_subs['Lead Source Description'] 
updated_subs['LEADREGIONS'] = updated_subs['Lead Regions'] 
updated_subs['LEADSTATUS'] = updated_subs['Lead Status'] 
updated_subs['EMAILOPTOUT'] = updated_subs['Email Opt Out'] 
updated_subs['OPTIN'] = updated_subs['Opt In']

In [ ]:
updated_subs = sub.reindex_cols(updated_subs, maillwizz_cols)
updated_subs

# bulk update subs
sub.update_data_subs(list['Test'], updated_subs)

In [ ]:
# cek email zoho yang tidak ada di mailwizz (akan dimasukan sebagai list subs baru, namun harus difiltrasi terlebih dahulu berdasrkan list email blacklist)
zoho_not_exist_in_mailzwizz = cut_zoho_df.loc[~cut_zoho_df['Email'].isin(load_maillwizz_df['EMAIL'])].copy()

# email zoho yang tidak ada di mailwizz akan difiltrasi dulu dari list email blacklist
input_email = zoho_not_exist_in_mailzwizz.loc[~zoho_not_exist_in_mailzwizz['Email'].isin(mailwizz_blacklist_df['Email'])].copy()
count_email_blacklist = zoho_not_exist_in_mailzwizz.loc[zoho_not_exist_in_mailzwizz['Email'].isin(mailwizz_blacklist_df['Email'])].shape[0]

input_email

In [ ]:
input_new_subs_zoho = sub.input_new_subs_from_zoho(list['Test'], input_email, 'Email', load_maillwizz_df, 'EMAIL')
deleted_mailwizz_df = sub.delete_maillwizz_not_exist_zoho(list['Test'], load_maillwizz_df, 'EMAIL', cut_zoho_df, 'Email')

count_error_input = input_new_subs_zoho.loc[input_new_subs_zoho['Status'] == 'Error'].shape[0]
count_success_input = input_new_subs_zoho.loc[input_new_subs_zoho['Status'] == 'Success'].shape[0]
count_deleted_mailwizz = deleted_mailwizz_df.shape[0]

In [ ]:
#input_new_subs_zoho.loc[input_new_subs_zoho['Status'] == 'Error'].to_csv(r'C:\Users\fajar\Documents\Python\Data\error.csv')

In [ ]:
count_final_api_data = (count_load_data_api-count_deleted_mailwizz)+count_success_input

In [ ]:
print("==========================================================")
print("Summary Data API")
print("==========================================================")
print("Jumlah Total Data Awal Maillwizz : "+str(count_load_data_api))
print("Jumlah Total Data Input Zoho: "+str(count_input_email_api))
print("Jumlah Email Mailwizz Exist di Email Zoho : "+str(count_email_mailwizz_exist))
print("Jumlah Email Mailwizz Tidak Exist di Email Zoho (Email Mailwizz Ini Akan Di Takeout Dari List Mailwizz) : "+str(count_deleted_mailwizz))
print("Jumlah Email Mailwizz Exist dan Terupdated : "+str(count_email_maillwizz_updated))
print("Jumlah Input Email Baru-Blacklisted Email (Dari Zoho ke Mailwizz) : "+str(count_email_blacklist))
print("Jumlah Input Email Baru Error (Dari Zoho ke Mailwizz) : "+str(count_error_input))
print("Jumlah Input Email Baru Success (Dari Zoho ke Mailwizz) : "+str(count_success_input))
print("Jumlah Data Akhir : "+str(count_final_api_data))

In [ ]:
data = {'Desc': [
                'Zoho Summary',
                'Jumlah Data Awal',
                'Jumlah Filter Awal', 
                'Jumlah Format Email Salah', 
                'Jumlah Setelah Clean Format Email Salah',
                'Jumlah Email Kosong',
                'Jumlah Setelah Clean Email Kosong',
                'Jumlah Email Duplikat',
                'Jumlah Setelah Clean Format Email Duplikat',
                'Jumlah Email OTA',
                'Jumlah Setelah Clean Email OTA',
                'Jumlah Wash Member',
                'Jumlah Setelah Wash Member',
                'Summary Data API',
                'Jumlah Total Data Awal Maillwizz',
                'Jumlah Total Data Input Zoho',
                'Jumlah Email Mailwizz Exist di Email Zoho',
                'Jumlah Email Mailwizz Tidak Exist di Email Zoho (Email mailwizz ini akan di takeout dari list mailwizz)',
                'Jumlah Email Mailwizz Exist dan Terupdated',
                'Jumlah Input Email Baru-Blacklisted Email',
                'Jumlah Input Email Baru Success (Dari Zoho ke Mailwizz)',
                'Jumlah Input Email Baru Error',
                'Jumlah Data Terakhir API'
                ],
        'Jumlah':[
                '=======================',
                str(count_data_awal),
                str(count_filter_awal),
                str(count_deleted_wrong_fromat),
                str(count_after_deleted_wrong_format),
                str(count_empty_email),
                str(count_after_dedup_empty_email),
                str(count_email_duplicate),
                str(count_after_dedup_duplicated_email),
                str(count_email_OTA),
                str(count_after_clean_OTA),
                str(count_wash_member),
                str(count_after_wash_member),
                '=======================',
                str(count_load_data_api),
                str(count_input_email_api),
                str(count_email_mailwizz_exist),
                str(count_deleted_mailwizz),
                str(count_email_maillwizz_updated),
                str(count_email_blacklist),
                str(count_success_input),
                str(count_error_input),
                str(count_final_api_data)
                ]
        }

# Create DataFrame
summary_df = pd.DataFrame(data)

now = datetime.now()

datetime = datetime.utcnow()

x = str(datetime).split(" ")

date = x[0]

# Print the output.
#summary_df.to_csv(r'C:\Users\fajar\Documents\Python\Data\summary_df'+' '+date+'.csv', index=False)